In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("/Users/mayankaggarwal/MNIST_data",one_hot=True,reshape=False)

Extracting /Users/mayankaggarwal/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/mayankaggarwal/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/mayankaggarwal/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/mayankaggarwal/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate=0.00001
n_output=10
epochs=10
batch_size=128
dropout=0.75
test_valid_size=256
save_file = './model.ckpt'

In [5]:
weights={
    "w1":tf.Variable(tf.truncated_normal([5,5,1,32])),
    "w2":tf.Variable(tf.truncated_normal([5,5,32,64])),
    "w3":tf.Variable(tf.truncated_normal([7*7*64,1024])),
    "w4":tf.Variable(tf.truncated_normal([1024,n_output]))
}

In [6]:
# print(weights['w3'].get_shape().as_list()[0])

In [7]:
biases={
    "b1":tf.random_normal([32]),
    "b2":tf.random_normal([64]),
    "b3":tf.random_normal([1024]),
    "b4":tf.random_normal([n_output])
}

In [8]:
def conv2d(x,w,b,s=1):
    output=tf.nn.conv2d(x,w,strides=[1,s,s,1],padding='SAME')
    output=tf.nn.bias_add(output,b)
    relu=tf.nn.relu(output)
    return output

In [9]:
def max_pool(x,k=2):
    output=tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
    return output

In [10]:
def conv_net(x,weights,biases,drop_out):
    conv1=conv2d(x,weights['w1'],biases['b1'])
    conv1=max_pool(conv1,k=2)
    
    conv2=conv2d(conv1,weights['w2'],biases['b2'])
    conv2=max_pool(conv2,k=2)
    
    fc=tf.reshape(conv2,[-1,7*7*64])
    fc=tf.add(tf.matmul(fc,weights['w3']),biases['b3'])
    fc=tf.nn.relu(fc)
    fc=tf.nn.dropout(fc,drop_out)
    
    output=tf.add(tf.matmul(fc,weights['w4']),biases['b4'])
    return output

In [11]:
x=tf.placeholder(tf.float32,shape=[None,28,28,1])
y=tf.placeholder(tf.float32,shape=[None,n_output])
keep_prob = tf.placeholder(tf.float32)

In [12]:
logits=conv_net(x,weights,biases,keep_prob)

In [13]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
correct_prediction=tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [15]:
saver = tf.train.Saver()

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x,batch_y=mnist.train.next_batch(batch_size) 
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:dropout})
            
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:256],
                y: mnist.validation.labels[:256],
                keep_prob: 1.})

            print('Epoch {}, Batch {}, Validation Accuracy: {}, Loss:{}'.format(
                epoch + 1,
                batch + 1,
                valid_acc,
                loss))
               
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:256],
        y: mnist.test.labels[:256],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
    saver.save(sess, save_file)

Epoch 1, Batch 1, Validation Accuracy: 0.140625, Loss:65176.6875
Epoch 1, Batch 2, Validation Accuracy: 0.125, Loss:46377.06640625
Epoch 1, Batch 3, Validation Accuracy: 0.078125, Loss:60751.39453125
Epoch 1, Batch 4, Validation Accuracy: 0.078125, Loss:44718.4140625
Epoch 1, Batch 5, Validation Accuracy: 0.11328125, Loss:45518.0859375
Epoch 1, Batch 6, Validation Accuracy: 0.19140625, Loss:32166.166015625
Epoch 1, Batch 7, Validation Accuracy: 0.1015625, Loss:46695.515625
Epoch 1, Batch 8, Validation Accuracy: 0.1015625, Loss:48634.1484375
Epoch 1, Batch 9, Validation Accuracy: 0.08984375, Loss:37024.609375
Epoch 1, Batch 10, Validation Accuracy: 0.078125, Loss:37572.4140625
Epoch 1, Batch 11, Validation Accuracy: 0.1171875, Loss:34751.3359375
Epoch 1, Batch 12, Validation Accuracy: 0.12890625, Loss:30336.0859375
Epoch 1, Batch 13, Validation Accuracy: 0.19921875, Loss:29654.70703125
Epoch 1, Batch 14, Validation Accuracy: 0.15625, Loss:40576.66796875
Epoch 1, Batch 15, Validation Acc

Epoch 1, Batch 117, Validation Accuracy: 0.49609375, Loss:12225.9833984375
Epoch 1, Batch 118, Validation Accuracy: 0.4765625, Loss:13416.943359375
Epoch 1, Batch 119, Validation Accuracy: 0.36328125, Loss:13812.63671875
Epoch 1, Batch 120, Validation Accuracy: 0.4453125, Loss:11046.375
Epoch 1, Batch 121, Validation Accuracy: 0.44140625, Loss:15565.5029296875
Epoch 1, Batch 122, Validation Accuracy: 0.54296875, Loss:13074.7939453125
Epoch 1, Batch 123, Validation Accuracy: 0.46484375, Loss:19942.716796875
Epoch 1, Batch 124, Validation Accuracy: 0.4375, Loss:10056.833984375
Epoch 1, Batch 125, Validation Accuracy: 0.41796875, Loss:12826.55859375
Epoch 1, Batch 126, Validation Accuracy: 0.4140625, Loss:18753.283203125
Epoch 1, Batch 127, Validation Accuracy: 0.4140625, Loss:20927.7421875
Epoch 1, Batch 128, Validation Accuracy: 0.47265625, Loss:12319.341796875
Epoch 1, Batch 129, Validation Accuracy: 0.27734375, Loss:20272.77734375
Epoch 1, Batch 130, Validation Accuracy: 0.3984375, Lo

Epoch 1, Batch 231, Validation Accuracy: 0.59765625, Loss:8792.900390625
Epoch 1, Batch 232, Validation Accuracy: 0.6328125, Loss:8941.17578125
Epoch 1, Batch 233, Validation Accuracy: 0.6171875, Loss:7823.34326171875
Epoch 1, Batch 234, Validation Accuracy: 0.640625, Loss:4645.23046875
Epoch 1, Batch 235, Validation Accuracy: 0.71875, Loss:6832.779296875
Epoch 1, Batch 236, Validation Accuracy: 0.68359375, Loss:5933.451171875
Epoch 1, Batch 237, Validation Accuracy: 0.609375, Loss:6278.0224609375
Epoch 1, Batch 238, Validation Accuracy: 0.71875, Loss:4829.76904296875
Epoch 1, Batch 239, Validation Accuracy: 0.5546875, Loss:5300.9501953125
Epoch 1, Batch 240, Validation Accuracy: 0.578125, Loss:7145.126953125
Epoch 1, Batch 241, Validation Accuracy: 0.66015625, Loss:5113.03271484375
Epoch 1, Batch 242, Validation Accuracy: 0.55078125, Loss:4943.25830078125
Epoch 1, Batch 243, Validation Accuracy: 0.71484375, Loss:8060.97265625
Epoch 1, Batch 244, Validation Accuracy: 0.65234375, Loss:1

Epoch 1, Batch 344, Validation Accuracy: 0.75, Loss:5055.193359375
Epoch 1, Batch 345, Validation Accuracy: 0.65625, Loss:4457.82080078125
Epoch 1, Batch 346, Validation Accuracy: 0.7265625, Loss:4942.83544921875
Epoch 1, Batch 347, Validation Accuracy: 0.71875, Loss:5973.271484375
Epoch 1, Batch 348, Validation Accuracy: 0.7265625, Loss:5376.6728515625
Epoch 1, Batch 349, Validation Accuracy: 0.6328125, Loss:6449.57177734375
Epoch 1, Batch 350, Validation Accuracy: 0.7265625, Loss:3577.736328125
Epoch 1, Batch 351, Validation Accuracy: 0.6640625, Loss:2742.37109375
Epoch 1, Batch 352, Validation Accuracy: 0.6875, Loss:5944.2939453125
Epoch 1, Batch 353, Validation Accuracy: 0.58984375, Loss:5099.1826171875
Epoch 1, Batch 354, Validation Accuracy: 0.73828125, Loss:6075.37109375
Epoch 1, Batch 355, Validation Accuracy: 0.71875, Loss:5463.296875
Epoch 1, Batch 356, Validation Accuracy: 0.70703125, Loss:6838.97802734375
Epoch 1, Batch 357, Validation Accuracy: 0.78125, Loss:4565.045898437

Epoch 2, Batch 28, Validation Accuracy: 0.76171875, Loss:5016.14013671875
Epoch 2, Batch 29, Validation Accuracy: 0.80078125, Loss:3677.172119140625
Epoch 2, Batch 30, Validation Accuracy: 0.75, Loss:7363.0693359375
Epoch 2, Batch 31, Validation Accuracy: 0.77734375, Loss:7077.9677734375
Epoch 2, Batch 32, Validation Accuracy: 0.8203125, Loss:5329.55029296875
Epoch 2, Batch 33, Validation Accuracy: 0.78125, Loss:2143.36669921875
Epoch 2, Batch 34, Validation Accuracy: 0.78125, Loss:3153.7529296875
Epoch 2, Batch 35, Validation Accuracy: 0.7578125, Loss:5432.25146484375
Epoch 2, Batch 36, Validation Accuracy: 0.7109375, Loss:5128.55078125
Epoch 2, Batch 37, Validation Accuracy: 0.7890625, Loss:3437.085693359375
Epoch 2, Batch 38, Validation Accuracy: 0.7421875, Loss:5101.6279296875
Epoch 2, Batch 39, Validation Accuracy: 0.71875, Loss:4299.6044921875
Epoch 2, Batch 40, Validation Accuracy: 0.7421875, Loss:4791.0751953125
Epoch 2, Batch 41, Validation Accuracy: 0.74609375, Loss:3374.8117

...

Epoch 9, Batch 369, Validation Accuracy: 0.9296875, Loss:1320.765380859375
Epoch 9, Batch 370, Validation Accuracy: 0.91015625, Loss:2504.49658203125
Epoch 9, Batch 371, Validation Accuracy: 0.90625, Loss:3026.32666015625
Epoch 9, Batch 372, Validation Accuracy: 0.92578125, Loss:951.360595703125
Epoch 9, Batch 373, Validation Accuracy: 0.9296875, Loss:2130.6689453125
Epoch 9, Batch 374, Validation Accuracy: 0.91796875, Loss:1204.9378662109375
Epoch 9, Batch 375, Validation Accuracy: 0.9296875, Loss:2203.25439453125
Epoch 9, Batch 376, Validation Accuracy: 0.91015625, Loss:1300.9576416015625
Epoch 9, Batch 377, Validation Accuracy: 0.9140625, Loss:1183.6163330078125
Epoch 9, Batch 378, Validation Accuracy: 0.9296875, Loss:671.5076904296875
Epoch 9, Batch 379, Validation Accuracy: 0.9140625, Loss:915.2600708007812
Epoch 9, Batch 380, Validation Accuracy: 0.91796875, Loss:1706.539306640625
Epoch 9, Batch 381, Validation Accuracy: 0.91796875, Loss:1356.1275634765625
Epoch 9, Batch 382, Val

Epoch 10, Batch 50, Validation Accuracy: 0.9140625, Loss:2777.60888671875
Epoch 10, Batch 51, Validation Accuracy: 0.8984375, Loss:1278.7105712890625
Epoch 10, Batch 52, Validation Accuracy: 0.92578125, Loss:1041.0150146484375
Epoch 10, Batch 53, Validation Accuracy: 0.92578125, Loss:603.517333984375
Epoch 10, Batch 54, Validation Accuracy: 0.90234375, Loss:1809.788818359375
Epoch 10, Batch 55, Validation Accuracy: 0.9140625, Loss:2193.826416015625
Epoch 10, Batch 56, Validation Accuracy: 0.9296875, Loss:1237.732177734375
Epoch 10, Batch 57, Validation Accuracy: 0.9296875, Loss:1516.31005859375
Epoch 10, Batch 58, Validation Accuracy: 0.91796875, Loss:496.2584228515625
Epoch 10, Batch 59, Validation Accuracy: 0.9296875, Loss:1792.272216796875
Epoch 10, Batch 60, Validation Accuracy: 0.90625, Loss:860.1341552734375
Epoch 10, Batch 61, Validation Accuracy: 0.91796875, Loss:1531.5914306640625
Epoch 10, Batch 62, Validation Accuracy: 0.93359375, Loss:1714.138427734375
Epoch 10, Batch 63, V

Epoch 10, Batch 159, Validation Accuracy: 0.9296875, Loss:924.2132568359375
Epoch 10, Batch 160, Validation Accuracy: 0.92578125, Loss:2017.8604736328125
Epoch 10, Batch 161, Validation Accuracy: 0.91015625, Loss:1288.466796875
Epoch 10, Batch 162, Validation Accuracy: 0.91796875, Loss:1435.6231689453125
Epoch 10, Batch 163, Validation Accuracy: 0.9375, Loss:1715.3272705078125
Epoch 10, Batch 164, Validation Accuracy: 0.92578125, Loss:929.4646606445312
Epoch 10, Batch 165, Validation Accuracy: 0.92578125, Loss:979.6896362304688
Epoch 10, Batch 166, Validation Accuracy: 0.953125, Loss:1122.10595703125
Epoch 10, Batch 167, Validation Accuracy: 0.91015625, Loss:2117.20654296875
Epoch 10, Batch 168, Validation Accuracy: 0.91015625, Loss:664.1656494140625
Epoch 10, Batch 169, Validation Accuracy: 0.9140625, Loss:1553.560302734375
Epoch 10, Batch 170, Validation Accuracy: 0.9296875, Loss:1333.3299560546875
Epoch 10, Batch 171, Validation Accuracy: 0.93359375, Loss:1622.96826171875
Epoch 10, 

Epoch 10, Batch 267, Validation Accuracy: 0.9296875, Loss:2232.12060546875
Epoch 10, Batch 268, Validation Accuracy: 0.921875, Loss:1246.7286376953125
Epoch 10, Batch 269, Validation Accuracy: 0.91796875, Loss:559.1777954101562
Epoch 10, Batch 270, Validation Accuracy: 0.90625, Loss:2073.310791015625
Epoch 10, Batch 271, Validation Accuracy: 0.90234375, Loss:1199.8736572265625
Epoch 10, Batch 272, Validation Accuracy: 0.9296875, Loss:2335.552734375
Epoch 10, Batch 273, Validation Accuracy: 0.9375, Loss:1235.129150390625
Epoch 10, Batch 274, Validation Accuracy: 0.921875, Loss:703.0823974609375
Epoch 10, Batch 275, Validation Accuracy: 0.91796875, Loss:1890.1580810546875
Epoch 10, Batch 276, Validation Accuracy: 0.9296875, Loss:1440.6669921875
Epoch 10, Batch 277, Validation Accuracy: 0.9375, Loss:1573.955810546875
Epoch 10, Batch 278, Validation Accuracy: 0.9296875, Loss:1580.1837158203125
Epoch 10, Batch 279, Validation Accuracy: 0.921875, Loss:980.624755859375
Epoch 10, Batch 280, Va

Epoch 10, Batch 376, Validation Accuracy: 0.90234375, Loss:948.7264404296875
Epoch 10, Batch 377, Validation Accuracy: 0.94140625, Loss:448.943115234375
Epoch 10, Batch 378, Validation Accuracy: 0.91015625, Loss:1243.560302734375
Epoch 10, Batch 379, Validation Accuracy: 0.9375, Loss:2225.65625
Epoch 10, Batch 380, Validation Accuracy: 0.9296875, Loss:1443.946533203125
Epoch 10, Batch 381, Validation Accuracy: 0.91796875, Loss:1838.86669921875
Epoch 10, Batch 382, Validation Accuracy: 0.9296875, Loss:1979.77392578125
Epoch 10, Batch 383, Validation Accuracy: 0.91015625, Loss:1113.135009765625
Epoch 10, Batch 384, Validation Accuracy: 0.9296875, Loss:2647.51318359375
Epoch 10, Batch 385, Validation Accuracy: 0.92578125, Loss:1362.0809326171875
Epoch 10, Batch 386, Validation Accuracy: 0.90625, Loss:2832.534423828125
Epoch 10, Batch 387, Validation Accuracy: 0.9375, Loss:293.8089904785156
Epoch 10, Batch 388, Validation Accuracy: 0.91796875, Loss:675.6355590820312
Epoch 10, Batch 389, Va